<a href="https://colab.research.google.com/github/Champioa00/LAB1/blob/master/Lab1Workingsession.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Anthony Champion ##
## IST 718 - LAB 1 ##
## 1/30/2022 ##
## Question: How can we recommend the best salary for our next head football coach? ##

In [2]:
## Load Required Packages ##
import pandas as pd
import io
import numpy as np 
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm
import statsmodels.formula.api as smf

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
## Load Coach Data Frame ##
coachdf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/Coaches9.csv')


In [4]:
## Explore Coach Data Frame ## 
print(coachdf.shape)
print(coachdf.info())
coachdf.head()

(129, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   School        129 non-null    object
 1   Conference    129 non-null    object
 2   Coach         129 non-null    object
 3   SchoolPay     129 non-null    object
 4   TotalPay      129 non-null    object
 5   Bonus         129 non-null    object
 6   BonusPaid     129 non-null    object
 7   AssistantPay  129 non-null    object
 8   Buyout        129 non-null    object
dtypes: object(9)
memory usage: 9.2+ KB
None


,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
0,Air Force,Mt. West,Troy Calhoun,885000,885000,247000,--,$0,--
1,Akron,MAC,Terry Bowden,"$411,000","$412,500","$225,000","$50,000",$0,"$688,500"
2,Alabama,SEC,Nick Saban,"$8,307,000","$8,307,000","$1,100,000","$500,000",$0,"$33,600,000"
3,Alabama at Birmingham,C-USA,Bill Clark,"$900,000","$900,000","$950,000","$165,471",$0,"$3,847,500"
4,Appalachian State,Sun Belt,Scott Satterfield,"$712,500","$712,500","$295,000","$145,000",$0,"$2,160,417"


The starting point of this analysis is the coaching dataset.  The first pass data exploration shows there are 9 columns and 129 unqiue observations.  Looking deeper, it's clear the numeric values all have a dollar sign ($) in front of them.  There also looks to be some values missing in BonusPaid and Buyout.  We will need to add some additional data points as well as perform some data transformation and data cleansing.

In [5]:
## Look at one conference and determine what variables need to be cleaned ##
coachdf[coachdf['Conference']=='SEC']

,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
2,Alabama,SEC,Nick Saban,"$8,307,000","$8,307,000","$1,100,000","$500,000",$0,"$33,600,000"
7,Arkansas,SEC,Chad Morris,"$3,500,000","$3,500,000","$1,000,000",--,$0,"$12,500,000"
10,Auburn,SEC,Gus Malzahn,"$6,700,000","$6,705,656","$1,400,000","$375,000",$0,"$32,143,750"
31,Florida,SEC,Dan Mullen,"$6,070,000","$6,070,000","$925,000",--,$0,"$12,000,000"
36,Georgia,SEC,Kirby Smart,"$6,603,600","$6,603,600","$1,150,000","$1,350,000",$0,"$27,917,500"
49,Kentucky,SEC,Mark Stoops,"$4,000,000","$4,013,600","$3,100,000","$300,000",$0,"$15,625,000"
55,LSU,SEC,Ed Orgeron,"$3,500,000","$3,500,000","$1,575,000","$100,000",$0,"$5,291,667"
66,Mississippi,SEC,Matt Luke,"$3,000,000","$3,000,000","$1,550,000",--,$0,$0
67,Mississippi State,SEC,Joe Moorhead,"$2,600,000","$2,600,000","$1,350,000",--,$0,$0
68,Missouri,SEC,Barry Odom,"$2,350,000","$2,350,000","$1,025,000","$375,000",$0,"$1,912,500"


All the dollar signs and commas need to be removed as well as the "--" data points converted to 0. Once completed we will start to introduce addtional datasets into the analysis.

In [6]:
## Select all the numeric columns ## 
num_columns = ['SchoolPay', 'TotalPay', 'Bonus', 'BonusPaid', 'AssistantPay', 'Buyout']

## Remove $, ",", and replace "--" with "0" ## 
for i in num_columns:
    coachdf[i] = coachdf[i].replace('[$,]', '', regex=True)
    coachdf[i] = coachdf[i].replace('[--]', '0', regex=True)
    coachdf[i] = coachdf[i].astype(float)

## Check Coach Dataset ## 
coachdf[coachdf['Conference']=='SEC']

,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0
7,Arkansas,SEC,Chad Morris,3500000.0,3500000.0,1000000.0,0.0,0.0,12500000.0
10,Auburn,SEC,Gus Malzahn,6700000.0,6705656.0,1400000.0,375000.0,0.0,32143750.0
31,Florida,SEC,Dan Mullen,6070000.0,6070000.0,925000.0,0.0,0.0,12000000.0
36,Georgia,SEC,Kirby Smart,6603600.0,6603600.0,1150000.0,1350000.0,0.0,27917500.0
49,Kentucky,SEC,Mark Stoops,4000000.0,4013600.0,3100000.0,300000.0,0.0,15625000.0
55,LSU,SEC,Ed Orgeron,3500000.0,3500000.0,1575000.0,100000.0,0.0,5291667.0
66,Mississippi,SEC,Matt Luke,3000000.0,3000000.0,1550000.0,0.0,0.0,0.0
67,Mississippi State,SEC,Joe Moorhead,2600000.0,2600000.0,1350000.0,0.0,0.0,0.0
68,Missouri,SEC,Barry Odom,2350000.0,2350000.0,1025000.0,375000.0,0.0,1912500.0


Add in addtional data sets to help with the analysis

In [7]:
## Load Records and Points Metrics by team and conference ##
recordsdf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/records.csv')

In [8]:
## Explore Coach Data Frame ## 
print(recordsdf.shape)
print(recordsdf.info())
recordsdf.head()

(130, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SCHOOL                     130 non-null    object 
 1   CONFERENCE                 130 non-null    object 
 2   W                          130 non-null    int64  
 3   L                          130 non-null    int64  
 4   PF                         130 non-null    int64  
 5   PA                         130 non-null    int64  
 6   AVG_POINTS_PER _GAME       130 non-null    float64
 7   DEF_ AVG_POINTS_PER _GAME  130 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 8.2+ KB
None


,SCHOOL,CONFERENCE,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
0,Cincinnati,AAC,13,1,516,236,36.9,16.9
1,Houston,AAC,12,2,508,286,36.3,20.4
2,UCF,AAC,9,4,478,333,36.8,25.6
3,SMU,AAC,8,4,461,341,38.4,28.4
4,East Carolina,AAC,7,5,356,315,29.7,26.3


In [9]:
## Clean up Numeric Columns ## 
## Select all the numeric columns ## 
recordssdf_num_columns = ['W', 'L', 'PF', 'PA']

## Remove $, ",", and replace "--" with "0" ## 
for i in recordssdf_num_columns:
    recordsdf[i] = recordsdf[i].astype(float)

## Check to see if the number format changed to float ##
print(recordsdf.shape)
print(recordsdf.info())

(130, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SCHOOL                     130 non-null    object 
 1   CONFERENCE                 130 non-null    object 
 2   W                          130 non-null    float64
 3   L                          130 non-null    float64
 4   PF                         130 non-null    float64
 5   PA                         130 non-null    float64
 6   AVG_POINTS_PER _GAME       130 non-null    float64
 7   DEF_ AVG_POINTS_PER _GAME  130 non-null    float64
dtypes: float64(6), object(2)
memory usage: 8.2+ KB
None


In [10]:
## Clean the Conference Data in the Coaches and Records ##

print(len(coachdf.Conference.unique()), coachdf.Conference.unique())
print(len(recordsdf.CONFERENCE.unique()), recordsdf.CONFERENCE.unique())

11 ['Mt. West' 'MAC' 'SEC' 'C-USA' 'Sun Belt' 'Pac-12' 'Ind.' 'Big 12' 'ACC'
 'AAC' 'Big Ten']
11 ['AAC' 'ACC' 'Big 12' 'Big Ten' 'C-USA' 'FBS Independent' 'MAC'
 'Mountain West' 'Pac-12' 'SEC' 'Sun Belt']


It looks like both the coach and record file have the same amount of conferences, lets look deeper and make sure they are fully aligned.

In [11]:
## Align the Conference Data ##

recordsdf_names = ['AAC','ACC','Big 12' ,'Big Ten' ,'C-USA', 'FBS Independent' ,'MAC' ,'Mountain West', 'Pac-12' ,'SEC' ,'Sun Belt']
new_name =        ['AAC' ,'ACC' ,'Big 12', 'Big Ten', 'C-USA' ,'Ind.', 'MAC', 'Mt. West' ,'Pac-12', 'SEC' ,'Sun Belt']  

recordsdf['CONFERENCE']=recordsdf['CONFERENCE'].replace(recordsdf_names, new_name, regex=False)


In [12]:
recordsdf_names

['AAC',
 'ACC',
 'Big 12',
 'Big Ten',
 'C-USA',
 'FBS Independent',
 'MAC',
 'Mountain West',
 'Pac-12',
 'SEC',
 'Sun Belt']

In [13]:
## Check to see if Conference change worked ## 
recordsdf[recordsdf['CONFERENCE']=='Ind.']


,SCHOOL,CONFERENCE,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
63,Notre Dame,Ind.,11.0,2.0,458.0,256.0,35.2,19.7
64,BYU,Ind.,10.0,3.0,430.0,322.0,33.1,24.8
65,Army West Point,Ind.,9.0,4.0,427.0,290.0,32.8,22.3
66,Liberty,Ind.,8.0,5.0,437.0,280.0,33.6,21.5
67,New Mexico St.,Ind.,2.0,10.0,269.0,485.0,22.4,40.4
68,Massachusetts,Ind.,1.0,11.0,196.0,517.0,16.3,43.1
69,UConn,Ind.,1.0,11.0,187.0,462.0,15.6,38.5



Looks like Conference for both the coaches and records data sets are the same, now lets load in some data set to help find our answer.  Next up is stadium infomration

In [14]:
## Load and Explore Stadium Dataset ##
stadiumdf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/Stadium.csv')

## Explore Stadium Data Frame ## 
print(stadiumdf.shape)
print(stadiumdf.info())
stadiumdf.head()

(131, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Stadium     131 non-null    object
 1   College     131 non-null    object
 2   Conference  131 non-null    object
 3   Capacity    131 non-null    int64 
 4   Opened      131 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 5.2+ KB
None


,Stadium,College,Conference,Capacity,Opened
0,Michigan Stadium,Michigan,Big Ten,107601,1927
1,Beaver Stadium,Penn State,Big Ten,106572,1960
2,Ohio Stadium,Ohio State,Big Ten,104944,1922
3,Kyle Field,Texas A&M,SEC,102733,1904
4,Neyland Stadium,Tennessee,SEC,102521,1921


The Stadium dataset looks to have the needed School and Conference fields, as well as 131 records.  Let's take a deeper dive into the Conference Names and align those to our coaches and records datasets.

In [15]:
## Clean the data and change numeric fields ##
## Clean up Numeric Columns ## 
## Select all the numeric columns ## 
stadiumdf_num_columns = ['Capacity']

# Change to numeric ## 
for i in stadiumdf_num_columns:
    stadiumdf[i] = stadiumdf[i].astype(float)

## Check to see if the number format changed to float ##
print(stadiumdf.shape)
print(stadiumdf.info())

(131, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Stadium     131 non-null    object 
 1   College     131 non-null    object 
 2   Conference  131 non-null    object 
 3   Capacity    131 non-null    float64
 4   Opened      131 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 5.2+ KB
None


In [16]:
## Align the Conference with Coaches and Records Dataframe ## 
print(len(coachdf.Conference.unique()), coachdf.Conference.unique())
print(len(recordsdf.CONFERENCE.unique()), recordsdf.CONFERENCE.unique())
print(len(stadiumdf.Conference.unique()), stadiumdf.Conference.unique())

11 ['Mt. West' 'MAC' 'SEC' 'C-USA' 'Sun Belt' 'Pac-12' 'Ind.' 'Big 12' 'ACC'
 'AAC' 'Big Ten']
11 ['AAC' 'ACC' 'Big 12' 'Big Ten' 'C-USA' 'Ind.' 'MAC' 'Mt. West' 'Pac-12'
 'SEC' 'Sun Belt']
11 ['Big Ten' 'SEC' 'Big 12' 'Pac 12' 'ACC' 'Independent' 'CUSA'
 'Mountain West' 'MAC' 'AAC' 'Sun Belt']


In [17]:
## All three have 11 unique Conferences, just need to rename a couple conferences ##

stadiumdf_names = ['AAC' ,'ACC' ,'Big 12', 'Big Ten', 'CUSA', 'Independent', 'MAC', 'Mountain West', 'Pac 12', 'SEC', 'Sun Belt']

stadiumdf['Conference']=stadiumdf['Conference'].replace(stadiumdf_names, new_name, regex=False)

print(len(coachdf.Conference.unique()), coachdf.Conference.unique())
print(len(recordsdf.CONFERENCE.unique()), recordsdf.CONFERENCE.unique())
print(len(stadiumdf.Conference.unique()), stadiumdf.Conference.unique())

11 ['Mt. West' 'MAC' 'SEC' 'C-USA' 'Sun Belt' 'Pac-12' 'Ind.' 'Big 12' 'ACC'
 'AAC' 'Big Ten']
11 ['AAC' 'ACC' 'Big 12' 'Big Ten' 'C-USA' 'Ind.' 'MAC' 'Mt. West' 'Pac-12'
 'SEC' 'Sun Belt']
11 ['Big Ten' 'SEC' 'Big 12' 'Pac-12' 'ACC' 'Ind.' 'C-USA' 'Mt. West' 'MAC'
 'AAC' 'Sun Belt']


Time to add the final dataset, Graduation meterics

In [18]:
## Load and Explore Graduation Dataset ##
graddf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/GradRate.csv')

## Explore Graduation Data Frame ## 
print(graddf.shape)
print(graddf.info())
graddf.head()

(256, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Cohort Year  256 non-null    int64 
 1   School       256 non-null    object
 2   Conference   256 non-null    object
 3   Sport        256 non-null    object
 4   State        256 non-null    object
 5   GSR          256 non-null    int64 
 6   FGR          256 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 14.1+ KB
None


,Cohort Year,School,Conference,Sport,State,GSR,FGR
0,2014,Abilene Christian University,Southland Conference,Football,TX,75,51
1,2014,University of Akron,Mid-American Conference,Football,OH,79,76
2,2014,Alabama A&M University,Southwestern Athletic Conf.,Football,AL,62,54
3,2014,Alabama State University,Southwestern Athletic Conf.,Football,AL,71,42
4,2014,University of Alabama,Southeastern Conference,Football,AL,86,65


In [19]:
## Clean the data and change numeric fields ##
## Clean up Numeric Columns ## 
## Select all the numeric columns ## 
graddf_num_columns = ['GSR' , 'FGR']

# Change to numeric ## 
for i in graddf_num_columns:
    graddf[i] = graddf[i].astype(float)

## Check to see if the number format changed to float ##
print(graddf.shape)
print(graddf.info())

(256, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Cohort Year  256 non-null    int64  
 1   School       256 non-null    object 
 2   Conference   256 non-null    object 
 3   Sport        256 non-null    object 
 4   State        256 non-null    object 
 5   GSR          256 non-null    float64
 6   FGR          256 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 14.1+ KB
None


In [23]:
## Align the Conference with Coaches and Records Dataframe ## 
print(len(graddf.Conference.unique()), graddf.Conference.unique())
new_name

25 ['Southland Conference' 'Mid-American Conference'
 'Southwestern Athletic Conf.' 'Southeastern Conference' 'Conference USA'
 'Colonial Athletic Association' 'Sun Belt Conference' 'Pac-12 Conference'
 'Ohio Valley Conference' 'Big South Conference' 'Big 12 Conference'
 'Mid-Eastern Athletic Conf.' 'Mountain West Conference'
 'Atlantic Coast Conference' 'Independent' 'The Ivy League'
 'Northeast Conference' 'Patriot League' 'Pioneer Football League'
 'Big Sky Conference' 'American Athletic Conference' 'Southern Conference'
 'Missouri Valley Football Conference' 'Big Ten Conference'
 'Horizon League']


['AAC',
 'ACC',
 'Big 12',
 'Big Ten',
 'C-USA',
 'Ind.',
 'MAC',
 'Mt. West',
 'Pac-12',
 'SEC',
 'Sun Belt']

In [45]:
## Transform Conference Column ##

grad_old_name = ['Southland Conference' ,'Mid-American Conference' ,'Southwestern Athletic Conf.' ,'Southeastern Conference' ,'Conference USA' ,'Colonial Athletic Association' ,
'Sun Belt Conference' ,'Pac-12 Conference' ,'Ohio Valley Conference' ,'Big South Conference' ,'Big 12 Conference' ,'Mid-Eastern Athletic Conf.' ,'Mountain West Conference' 
,'Atlantic Coast Conference' ,'Independent' ,'The Ivy League' ,'Northeast Conference' ,'Patriot League' ,'Pioneer Football League' ,'Big Sky Conference' ,'American Athletic Conference'
,'Southern Conference', 'Missouri Valley Football Conference' ,'Big Ten Conference', 'Horizon League']

grad_new_name = ['SLC', 'MAC', 'SWAC', 'SEC', 'C-USA', 'CAA', 'Sun Belt', 'Pac-12', 'OVC', 'Big South', 'Big 12', 'MEAC', 'Mt. West', 'ACC', 'Ind.','IVY', 'NEC', 'PAT', 'PFL', 
                 'Big Sky', 'AAC', 'SOCON', 'MVFC', 'Big Ten' , 'HOL']


graddf['Conference'] = graddf['Conference'].replace(grad_old_name, grad_new_name, regex=False)

## Check to see if Conference change worked ## 
graddf[graddf['Conference']=='IVY']

,Cohort Year,School,Conference,Sport,State,GSR,FGR
23,2014,Brown University,IVY,Football,RI,95.0,0.0
45,2014,Columbia University-Barnard College,IVY,Football,NY,99.0,0.0
47,2014,Cornell University,IVY,Football,NY,97.0,0.0
48,2014,Dartmouth College,IVY,Football,NH,100.0,0.0
76,2014,Harvard University,IVY,Football,MA,98.0,0.0
162,2014,University of Pennsylvania,IVY,Football,PA,91.0,90.0
166,2014,Princeton University,IVY,Football,NJ,99.0,100.0
244,2014,Yale University,IVY,Football,CT,98.0,0.0
